In [2]:
import pandas as pd
import numpy as np
import bokeh.plotting as bp
import matplotlib.pyplot as plt
%matplotlib inline

#### Para empezar se probará con los pares EUR/USD y GBP/USD para predecir la señal EUR/USD

In [3]:
#Se lee la señal correspondiente a GBP vs USD para el mes de enero de 2018
data_GBPvsUSD = pd.read_csv("../data/TrueFX/GBP-USD/datos_procesados_5T_GBPUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

#Se lee la señal correspondiente a EUR vs USD para el mes de enero de 2018
data_EURvsUSD = pd.read_csv("../data/TrueFX/EUR-USD/datos_procesados_5T_EURUSD-2018-01.csv", index_col=0
                            ,infer_datetime_format=True, parse_dates=True )

In [48]:
%run ../code/EUtilities.py
%run ../code/PNLEstimatorWrapper.py
EU = EUtilities()

In [12]:
print('tamaño de la señal EUR/USD', data_EURvsUSD.shape)
print('tamaño de la señal GBP/USD', data_GBPvsUSD.shape)

tamaño de la señal EUR/USD (6361, 2)
tamaño de la señal GBP/USD (6360, 2)


para poder utilizar la función "multi_signal" se deben cumplir ciertas caracteristicas, entre ellas, que los dataframe a utilizar tenga su columna llamada "bid"...

In [13]:
#se construye el dataset EUR/USD con las dos señales en el, para su posterior uso
data_EURvsUSD = EU.multi_signal(data_EURvsUSD,data_GBPvsUSD)

In [14]:
data_EURvsUSD[:10]

,bid,ask,bidnew
2018-01-02 00:00:00,1.20015,1.20110,1.20015
2018-01-02 00:05:00,1.20047,1.20084,1.34940
2018-01-02 00:10:00,1.20082,1.20126,1.34895
2018-01-02 00:15:00,1.20035,1.20091,1.34945
2018-01-02 00:20:00,1.20049,1.20085,1.34953
2018-01-02 00:25:00,1.20047,1.20085,1.34953
2018-01-02 00:30:00,1.20079,1.20113,1.34963
2018-01-02 00:35:00,1.20135,1.20155,1.35042
2018-01-02 00:40:00,1.20143,1.20158,1.34983
2018-01-02 00:45:00,1.20149,1.20163,1.34950


procedemos a construir ambos dataset por separados para luego unirlos

In [20]:
#los datos del par EUR/USD 
X_EURUSD, y_EURUSD, bt_EURUSD = EU.build_dataset(data_EURvsUSD,bid_col='bid',window=7,binary_target=True,PNL=True
                                             ,delete_constant_values=True)
#los datos del par GBP/USD para luego unirlos y formar los datos X para entrenar
X_GBPUSD,_ = EU.build_dataset(data_EURvsUSD,bid_col='bidnew',window=7,binary_target=False
                              ,delete_constant_values=True)

In [21]:
X_EURUSD.shape, X_GBPUSD.shape

((6232, 9), (6237, 7))

debido a que son diferentes señales, al habilitar la opción de <i>"delete_constant_values"</i> en la función <i>"build_dataset"</i> esto hace que las señales resultantes no sean iguales en tamaño, por ende para entrenar se cogeran los valores de los indices de la señal a medir, si este no está en la otra señal, se coloca lo de la señal EUR/USD

In [22]:
X_GBPUSD = X_GBPUSD.loc[X_EURUSD.index]
X_EURUSD.shape, X_GBPUSD.shape

((6232, 9), (6232, 7))

debido a que si el valor del indice del par EUR/USD no se encuentra en el par GBP/USD entonces este genera valores nan, estos valores se cambian por el valor que hay en el par EUR/USD

In [23]:
#para cambiar los valores NaN por los que hay en la señal EUR/USD
X_GBPUSD = X_GBPUSD.fillna(X_EURUSD,axis=0)

se unen para entrenar la multiseñal

In [43]:
window=7
for i in X_GBPUSD.columns:
    X_EURUSD.insert(loc=window+int(i),column=window+int(i),value=X_GBPUSD.iloc[:,i])

In [44]:
X_EURUSD.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,PNL_0,PNL_1
2018-01-02 00:30:00,1.20015,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20015,1.34940,1.34895,1.34945,1.34953,1.34953,1.34963,-0.00076,0.00022
2018-01-02 00:35:00,1.20047,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.34940,1.34895,1.34945,1.34953,1.34953,1.34963,1.35042,-0.00023,-0.00012
2018-01-02 00:40:00,1.20082,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.34895,1.34945,1.34953,1.34953,1.34963,1.35042,1.34983,-0.00020,-0.00009
2018-01-02 00:45:00,1.20035,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.34945,1.34953,1.34953,1.34963,1.35042,1.34983,1.34950,-0.00007,-0.00030
2018-01-02 00:50:00,1.20049,1.20047,1.20079,1.20135,1.20143,1.20149,1.20133,1.34953,1.34953,1.34963,1.35042,1.34983,1.34950,1.34968,-0.00028,-0.00017


#### probando la multiseñal con el periodo de 5T

In [50]:
from sklearn.naive_bayes import GaussianNB
    
y = bt_EURUSD
n_bdtrain = 4
n_bdtest = 1
mday = 1
#para indexar el dataframe por las fechas
dates = np.unique(X_EURUSD.index.date)[n_bdtrain:]
est_GNB = PNLEstimatorWrapper(GaussianNB(), PNL_column=['PNL_0','PNL_1'])
result_GNB = EU.step_validation(est_GNB, X_EURUSD, y, EU.v_split(X_EURUSD,n_bdtrain,n_bdtest,mday))
result_GNB, predict_GNB = EU.redim(result_GNB)
print('--------- GaussianNB 5m ----------------')
print('average PNL : ', np.mean(result_GNB[:,0]))
print('average buy: ', np.mean(result_GNB[:,1]))
print('average sell: ', np.mean(result_GNB[:,2]))
print('longest_buy: ', np.max(result_GNB[:,3]))
print('longest_sell: ', np.max(result_GNB[:,4]))
print('%correct_1: ', np.mean(result_GNB[:,5]))
print('%correct_0: ', np.mean(result_GNB[:,6]))
print('%correct_3: ', np.mean(result_GNB[:,7]))
print('%correct_2: ', np.mean(result_GNB[:,8]))
print('% correct total: ', np.mean(result_GNB[:,5]+result_GNB[:,6]+result_GNB[:,7]+result_GNB[:,8]))
print('-------------------------------------')
df_GNB = pd.DataFrame(result_GNB, columns=list(['PNL','buys','sells','longest_buys','longest_sells'
                                                ,'%correct_1','%correct_0','%correct_3','%correct_2']), index=dates)
print(df_GNB)
df_GNB.to_csv('../resultados/TrueFX/EUR-USD/MultiSeñal/GaussianNB/pnl_5T_EUR-USD_GBP-USD_GNB_NC4_wsize7_2018-01.csv')

--------- GaussianNB 5m ----------------
average PNL :  -0.0038047826087
average buy:  109.695652174
average sell:  112.47826087
longest_buy:  267.0
longest_sell:  288.0
%correct_1:  0.127552755757
%correct_0:  0.0946245566433
%correct_3:  0.0204626330719
%correct_2:  0.0556370631486
% correct total:  0.298277008621
-------------------------------------
                PNL   buys  sells  longest_buys  longest_sells  %correct_1  \
2018-01-06 -0.00057    1.0    0.0           1.0            0.0    0.000000   
2018-01-08 -0.01514  279.0    0.0          87.0           39.0    0.301075   
2018-01-09 -0.01132  283.0    0.0          77.0            1.0    0.416961   
2018-01-10 -0.00928  277.0    4.0         160.0            1.0    0.380783   
2018-01-11  0.00072  182.0  100.0          59.0          102.0    0.273050   
2018-01-12  0.00000    9.0  271.0           0.0          280.0    0.000000   
2018-01-13 -0.00138    1.0    0.0           1.0            0.0    0.000000   
2018-01-15 -0.00548 